In [ ]:
import ccxt
import pandas as pd
import time

def fetch_funding_rate_history(exchange, symbol, since=None, until=None, limit=1000):
    """
    Fetch funding rate data with pagination for Binance Futures (USDT-M).
    """
    all_rates = []
    while True:
        # Fetch one batch
        batch = exchange.fetch_funding_rate_history(
            symbol=symbol,
            since=since,
            limit=limit,
            params={"endTime": until} if until else {}
        )
        if not batch:
            break

        all_rates.extend(batch)

        last_ts = batch[-1]["timestamp"]
        since = last_ts + 1 

        if until and since >= until:
            break

        time.sleep(exchange.rateLimit / 1000)

        # stop if fewer than limit results (end of data)
        if len(batch) < limit:
            break

    return all_rates


# Setup Binance Futures (USDT-M)
exchange = ccxt.binance({
    "options": {"defaultType": "future"},
})

# 26 chosen assets
symbols = [
    "AAVE/USDT", "ADA/USDT", "APT/USDT", "ARB/USDT", "ATOM/USDT",
    "AVAX/USDT", "BCH/USDT", "BNB/USDT", "BTC/USDT", "DOGE/USDT",
    "DOT/USDT", "ENA/USDT", "ETC/USDT", "ETH/USDT", "HBAR/USDT",
    "LINK/USDT", "LTC/USDT", "NEAR/USDT", "SOL/USDT", "SUI/USDT",
    "TON/USDT", "TRX/USDT", "UNI/USDT", "WLD/USDT", "XLM/USDT", "XRP/USDT"
]

# Time window: Feb 2024 → Apr 2025
since = exchange.parse8601("2024-05-01T00:00:00Z")
until = exchange.parse8601("2025-04-30T23:59:00Z")

# Iterate over symbols
for symbol in symbols:
    print(f"Fetching funding rates for {symbol}...")
    try:
        data = fetch_funding_rate_history(exchange, symbol, since=since, until=until)

        if not data:
            print(f"No funding data for {symbol}")
            continue

        df = pd.DataFrame(data)
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
        df["funding_rate"] = df["fundingRate"]

        df = df[["datetime", "symbol", "funding_rate"]].sort_values("datetime")

        fname = f"{symbol.replace('/', '')}_funding_bin_futures.parquet"
        df.to_parquet(fname)
        print(f"✅ Saved {symbol} → {fname}, {len(df)} rows")

    except Exception as e:
        print(f"⚠️ Error fetching {symbol}: {e}")
        time.sleep(5)


Fetching funding rates for AAVE/USDT...
✅ Saved AAVE/USDT → AAVEUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for ADA/USDT...
✅ Saved ADA/USDT → ADAUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for APT/USDT...
✅ Saved APT/USDT → APTUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for ARB/USDT...
✅ Saved ARB/USDT → ARBUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for ATOM/USDT...
✅ Saved ATOM/USDT → ATOMUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for AVAX/USDT...
✅ Saved AVAX/USDT → AVAXUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for BCH/USDT...
✅ Saved BCH/USDT → BCHUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for BNB/USDT...
✅ Saved BNB/USDT → BNBUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for BTC/USDT...
✅ Saved BTC/USDT → BTCUSDT_funding_bin_futures.parquet, 1095 rows
Fetching funding rates for DOGE/USDT...
✅ Save

In [ ]:
import ccxt
import pandas as pd
import time

def fetch_ohlcv_history(exchange, symbol, timeframe="1m", since=None, until=None, limit=1500):
    """
    Fetch OHLCV (candlestick) data with pagination.
    """
    all_candles = []
    while True:
        batch = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
        if not batch:
            break

        all_candles.extend(batch)

        last_ts = batch[-1][0]
        since = last_ts + 1

        if until and since >= until:
            break

        # respect Binance rate limits
        time.sleep(exchange.rateLimit / 1000)

    return all_candles


# Setup Binance Futures (USDT-M) 
exchange = ccxt.binance({"options": {"defaultType": "future"}})

# The 26 symbols you want to keep (quoted in USDT)
symbols = [
    "AAVE/USDT", "ADA/USDT", "APT/USDT", "ARB/USDT", "ATOM/USDT",
    "AVAX/USDT", "BCH/USDT", "BNB/USDT", "BTC/USDT", "DOGE/USDT",
    "DOT/USDT", "ENA/USDT", "ETC/USDT", "ETH/USDT", "HBAR/USDT",
    "LINK/USDT", "LTC/USDT", "NEAR/USDT", "SOL/USDT", "SUI/USDT",
    "TON/USDT", "TRX/USDT", "UNI/USDT", "WLD/USDT", "XLM/USDT", "XRP/USDT"
]

# Time window: Feb 2024 → Apr 2025
since = exchange.parse8601("2024-02-01T00:00:00Z")
until = exchange.parse8601("2025-04-30T23:59:00Z")

# Iterate over symbols
for symbol in symbols:
    print(f"Fetching {symbol}...")
    ohlcv = fetch_ohlcv_history(exchange, symbol, timeframe="1m", since=since, until=until)

    df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")

    # Save each symbol to Parquet
    fname = f"{symbol.replace('/', '')}_1m_bin_futures.parquet"
    df.to_parquet(fname)
    print(f"Saved {symbol} → {fname}, {len(df)} rows")


Fetching AAVE/USDT...
Saved AAVE/USDT → AAVEUSDT_1m_bin_futures.parquet, 655500 rows
Fetching ADA/USDT...
Saved ADA/USDT → ADAUSDT_1m_bin_futures.parquet, 655500 rows
Fetching APT/USDT...
Saved APT/USDT → APTUSDT_1m_bin_futures.parquet, 655500 rows
Fetching ARB/USDT...
Saved ARB/USDT → ARBUSDT_1m_bin_futures.parquet, 655500 rows
Fetching ATOM/USDT...
Saved ATOM/USDT → ATOMUSDT_1m_bin_futures.parquet, 655500 rows
Fetching AVAX/USDT...
Saved AVAX/USDT → AVAXUSDT_1m_bin_futures.parquet, 655500 rows
Fetching BCH/USDT...
Saved BCH/USDT → BCHUSDT_1m_bin_futures.parquet, 655500 rows
Fetching BNB/USDT...
Saved BNB/USDT → BNBUSDT_1m_bin_futures.parquet, 655500 rows
Fetching BTC/USDT...
Saved BTC/USDT → BTCUSDT_1m_bin_futures.parquet, 655500 rows
Fetching DOGE/USDT...
Saved DOGE/USDT → DOGEUSDT_1m_bin_futures.parquet, 655500 rows
Fetching DOT/USDT...
Saved DOT/USDT → DOTUSDT_1m_bin_futures.parquet, 655500 rows
Fetching ENA/USDT...
Saved ENA/USDT → ENAUSDT_1m_bin_futures.parquet, 567000 rows
Fetc